In [ ]:
#|default_exp web_config

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

# Manage web assets

In [ ]:
#| export

from pathlib import Path
from shutil import copy

from fastcore.all import in_ipython

from clip_plot import utils
from clip_plot.utils import get_version

In [ ]:
#|export

def get_clip_plot_root() -> Path:
    # ipython doesn't have __file__ attribute
    if in_ipython():
        return Path(utils.__file__).parents[1]
    else:
        return Path(__file__).parents[1]

In [ ]:
#| export

def replace_web_strings(web_dir: Path, from_to: dict):
    '''sequentially replace strings based on dict'''
    for i in ["index.html", "assets/js/tsne.js"]:
        f = web_dir / i
        t = f.read_text()
        for k, v in from_to.items():
            t = t.replace(k, v)
        f.write_text(t)


In [ ]:
#| export

def byo_logo(web_dir: Path, logo_path: Path):
    '''copy logo into web folder and replace reference in index.html'''
    logo_dest = web_dir / "assets/images" / logo_path.name
    copy(logo_path, logo_dest)
    replace_web_strings(web_dir, {"cp_logo.png": logo_path.name,
                                  "clip-plot logo": "Custom logo"
                               })

In [ ]:
#| export

def copy_web_assets(output_dir: Path, tagline: str | None, logo: Path | None) -> None:
    """
    Copy the /web directory from the clipplot source to the users cwd.
    Copies version number into assets.
    """
    copy_root_dir = get_clip_plot_root()
    src = copy_root_dir / "clip_plot/web"

    # resolve will handle cases with ../ in the path
    dest = Path.cwd() / output_dir.resolve()
    utils.copytree_agnostic(src.as_posix(), dest.as_posix())

    replace_web_strings(dest, {"VERSION_NUMBER": get_version()})

    if tagline:
        replace_web_strings(dest, {"Image Fields in a Local Collection": tagline})

    if logo:
        logo_resolved = Path.cwd() / logo.resolve()
        byo_logo(dest, logo_resolved)


In [ ]:
#|hide
import nbdev

nbdev.nbdev_export()